In [1]:
!pip install flask transformers torch


Defaulting to user installation because normal site-packages is not writeable


In [2]:
pip install flask python-docx PyMuPDF sumy nltk


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from nltk.tokenize import sent_tokenize


In [3]:
!pip install flask python-docx PyMuPDF


Defaulting to user installation because normal site-packages is not writeable


In [4]:
pip install sumy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask, request, render_template_string
from werkzeug.utils import secure_filename
import os, re, fitz, docx, logging
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
ALLOWED_EXTENSIONS = {'.txt', '.pdf', '.docx'}
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
class LexRankSummarizerWrapper:
    def __init__(self, language='english'):
        self.language = language
        self.summarizer = LexRankSummarizer()
    def summarize(self, text, sentence_count=5):
        parser = PlaintextParser.from_string(text, Tokenizer(self.language))
        summary = self.summarizer(parser.document, sentence_count)
        return ' '.join(str(sentence) for sentence in summary)

def synonym_replace(text, level=1):
    if level == 0:
        return text
    words = word_tokenize(text)
    new_words = []
    for word in words:
        if not word.isalpha():
            new_words.append(word)
            continue
        syns = wordnet.synsets(word)
        if syns:
            lemmas = syns[0].lemmas()
            if len(lemmas) > level:  
                new_word = lemmas[level].name().replace('_', ' ')
                if new_word.lower() != word.lower():
                    new_words.append(new_word)
                else:
                    new_words.append(word)
            else:
                new_words.append(word)
        else:
            new_words.append(word)

    return ' '.join(new_words)
def read_file(filepath, ext):
    try:
        if ext == '.txt':
            with open(filepath, 'r', encoding='utf-8') as f:
                return f.read()
        elif ext == '.pdf':
            text = ''
            with fitz.open(filepath) as doc:
                for page in doc:
                    text += page.get_text()
            return text
        elif ext == '.docx':
            doc = docx.Document(filepath)
            return '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        logging.error(f"Error reading file: {e}", exc_info=True)
        return ''
HTML_PAGE = '''
<!DOCTYPE html>
<html>
<head>
    <title>Quantum AI Summarizer</title>
    <style>
        body { font-family: Arial; background-color: #0d1117; color: #e6edf3; padding: 20px; text-align: center; }
        h2 { color: #00bcd4; }
        form { background: #161b22; padding: 20px; border-radius: 10px; margin-bottom: 20px; display: inline-block; }
        input, select { margin: 10px 0; padding: 10px; width: 100%; border-radius: 5px; }
        .summary-box { background: #161b22; padding: 20px; border: 1px solid #21262d; border-radius: 10px; max-width: 800px; margin: auto; white-space: pre-wrap; }
        .error { color: #ff6b6b; }
    </style>
</head>
<body>
    <h2>Quantum AI Summarizer</h2>
    <form method="POST" enctype="multipart/form-data">
        <input type="file" name="text_file" accept=".txt,.pdf,.docx" required><br>
        <label>Number of Sentences:</label>
        <select name="num_sentences">
            {% for i in range(1, 11) %}
                <option value="{{ i }}">{{ i }}</option>
            {% endfor %}
        </select>
        <label>Synonym Level:</label>
        <select name="syn_level">
            <option value="0">None</option>
            <option value="1">Light</option>
            <option value="2">Medium</option>
            <option value="3">High</option>
        </select><br>
        <input type="submit" value="Summarize">
    </form>
    {% if summary %}
        <div class="summary-box">{{ summary }}</div>
    {% elif error %}
        <div class="error">{{ error }}</div>
    {% endif %}
</body>
</html>
'''
@app.route("/", methods=["GET", "POST"])
def upload_file():
    summary = ''
    error = ''
    if request.method == "POST":
        file = request.files.get("text_file")
        num_sentences = int(request.form.get("num_sentences", 3))
        syn_level = int(request.form.get("syn_level", 0))

        if not file or file.filename == '':
            error = "No file selected."
            return render_template_string(HTML_PAGE, summary=summary, error=error)

        filename = secure_filename(file.filename)
        ext = os.path.splitext(filename)[1].lower()
        if ext not in ALLOWED_EXTENSIONS:
            error = "Unsupported file type."
            return render_template_string(HTML_PAGE, summary=summary, error=error)

        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        content = read_file(filepath, ext)
        os.remove(filepath)

        if not content.strip():
            error = "File is empty or unreadable."
        else:
            summarizer = LexRankSummarizerWrapper()
            extracted = summarizer.summarize(content, num_sentences)
            summary = synonym_replace(extracted, level=syn_level)

    return render_template_string(HTML_PAGE, summary=summary, error=error)
if __name__ == "__main__":
    app.run(host="127.0.0.1", port=5000, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Jul/2025 20:23:25] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jul/2025 20:23:41] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jul/2025 20:24:00] "POST / HTTP/1.1" 200 -


In [1]:
from rouge_score import rouge_scorer

ref = "The quick brown fox jumps over the lazy dog."
gen = "A fast brown fox leaps over a lazy dog."

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(ref, gen)
print(scores)


{'rouge1': Score(precision=0.5555555555555556, recall=0.5555555555555556, fmeasure=0.5555555555555556), 'rougeL': Score(precision=0.5555555555555556, recall=0.5555555555555556, fmeasure=0.5555555555555556)}
